In [1]:
import os
import openai
import pandas as pd
import json
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [2]:
# OpenAI API 키 설정
openai.api_key = "개인키작성"
os.environ['OPENAI_API_KEY'] = "개인키작성"

In [11]:
def text_generator(prompt, model="gpt-4o", temperature=0):
    messages = [
        {"role": "system", "content": "질문에 오류 혹은 잘못된 정보가 있는지 확인하고, 있다면 이것을 지적하고 수정한다."},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    answer = response.choices[0].message.content
    return answer

In [21]:
def get_arg_answer(query, verbose=False):
    prompt = f"""
    당신은 데이터 프레임에서 쿼리한 데이터에 대한 질의 응답 봇입니다. 
    특히, 국가 정보 데이터베이스를 가지고 있는 전문가입니다.
    
    df 열과 열 설명은 다음과 같습니다: 
        국가코드: 각 국가를 식별하기 위한 고유한 코드입니다.
        국가한글명: 각 국가의 한글 표기명입니다.
        국가영문명: 각 국가의 영문 표기명입니다.
        국가대륙코드: 각 국가가 속한 대륙을 나타내는 코드로 대륙명을 나타냅니다.
        
    앞으로 문장이 들어오면 국가코드, 국가한글명, 국가영문명, 국가대륙코드와 관련된 추가 질문을 10개 생성해주세요.
    예시를 제공해드립니다. 예시 외의 질문들을 추가로 작성해줘야 합니다.
    문장의 구분은 '\n'로 진행해야 합니다.
    문장들을 구분하는데 있어서 "1."와 같이 숫자가 있다면 제외해주세요.
    ex) 1. 러시아의 국가한글명은 무엇인가요? -> 러시아의 국가한글명은 무엇인가요?
    
    output은 꼭 아래의 예시 템플릿을 따라주세요.
    
    예시
    input : 국가코드, 국가한글명, 국가영어명, 국가대륙코드에 대한 csv 파일
    output : 아르메니아의 국가코드는 무엇인가요?\n아제르바이잔의 국가영문명은 무엇인가요?\n방글라데시는 어느 대륙에 속해있나요?\n'BN'은 어느나라의 국가코드인가요?\n부탄의 국가코드, 국가대륙코드는 각각 무엇인가요?

    질문은 output에 속하는 내용만 포함하면 됩니다.
    또한, 아래와 같은 형식은 -> 다음 형식으로 수정부탁드립니다.
    ex) \"IN\"은 어느나라의 국가코드인가요? -> 'IN'은 어느나라 국가코드인가요?

    또한, "아래는 제공된 데이터에 기반한 추가 질문 10개입니다:" 이와 같은 설명 문구는 제거해주세요.
    
    이제 당신이 생성해야할 글을 제공해드리겠습니다.
    {query}
    """
    if verbose:
        print(f'Prompt: {prompt}')
    answer = text_generator(prompt)
    return answer.split("\n")

In [24]:
def generate_qa_from_csv_with_langchain(csv_path, verbose=False):
    # CSV 파일을 데이터프레임으로 로드
    df = pd.read_csv(csv_path, encoding='cp949')

    prefix = """
    당신은 데이터 프레임에서 쿼리한 데이터에 대한 질의 응답 봇입니다. 
    특히, 국가 정보 데이터베이스를 가지고 있는 전문가입니다.
    
    df 열과 열 설명은 다음과 같습니다: 
        국가코드: 각 국가를 식별하기 위한 고유한 코드입니다.
        국가한글명: 각 국가의 한글 표기명입니다.
        국가영문명: 각 국가의 영문 표기명입니다.
        국가대륙코드: 각 국가가 속한 대륙을 나타내는 코드로 대륙명을 나타냅니다.
    """
    
    # 답변 제공할  고려해야 할 추가 지침
    suffix = """
    출력물의 최대 문자 길이는 300자로 제한하겠습니다.
    답변을 최대 문자 길이 내로 축약해서 전달해주면 됩니다.

    답변은 예시와 같은 형식으로 제공해주세요. 
    input : 아르메니아의 국가코드는 무엇인가요?\n아제르바이잔의 국가영문명은 무엇인가요?\n방글라데시는 어느 대륙에 속해있나요?\n'BN'은 어느나라의 국가코드인가요?\n부탄의 국가코드, 국가대륙코드는 각각 무엇인가요?
    output : 아르메니아의 국가코드는 'AM'입니다.\n아제르바이잔의 국가영문명은 'AZERBAIJAN'입니다.\n방글라데시의 국가대륙코드는 '아시아'로 아시아에 속해있습니다.\n'BN'은 브루나이의 국가코드입니다.\n부탄의 국가코드는 'BT', 국가대륙코드는 '아시아'입니다.

    답변은 output에 속하는 내용만 포함하면 됩니다.

    - 주의사항 1: 한국어로 들어온 질문에는 한국어로 대답해야 합니다.
    - 주의사항 2: 단답형으로 대답하지 말아주세요!
    ex) "QUESTION": "키르기스스탄의 국가영문명은 무엇인가요?" -> "ANSWER": "KYRGYZSTAN" 일 경우
        "키르기스스탄의 국가영문명은 KYRGYZSTAN 입니다." 라고 답변할 것.
        
    """

    # LangChain의 OpenAI 에이전트 생성
    agent = create_pandas_dataframe_agent(ChatOpenAI(temperature=0), df, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, allow_dangerous_code=True)

    # 데이터프레임의 정보를 바탕으로 질문 생성
    questions = get_arg_answer(df.to_string(index=False), verbose=verbose)

    # 각 질문에 대해 에이전트를 사용해 답변 생성
    answers = []
    for question in questions:
        response = agent.run(question)
        answers.append(response)

    # JSON 형식으로 변환
    qa_pairs = [{"QUESTION": q.strip(), "ANSWER": a.strip()} for q, a in zip(questions, answers)]

    return qa_pairs

In [25]:
csv_path = "D:/[24]ICT_Practice/practice_file/code.csv"
qa_pairs = generate_qa_from_csv_with_langchain(csv_path)

print(json.dumps(qa_pairs, ensure_ascii=False, indent=2))

[
  {
    "QUESTION": "아르메니아의 국가코드는 무엇인가요?",
    "ANSWER": "The country code for Armenia is AM."
  },
  {
    "QUESTION": "아제르바이잔의 국가영문명은 무엇인가요?",
    "ANSWER": "AZERBAIJAN"
  },
  {
    "QUESTION": "방글라데시는 어느 대륙에 속해있나요?",
    "ANSWER": "Asia"
  },
  {
    "QUESTION": "'BN'은 어느 나라의 국가코드인가요?",
    "ANSWER": "'BN'은 브루나이의 국가코드입니다."
  },
  {
    "QUESTION": "부탄의 국가코드, 국가대륙코드는 각각 무엇인가요?",
    "ANSWER": "국가코드는 BT이고, 국가대륙코드는 아시아입니다."
  },
  {
    "QUESTION": "중국의 국가영문명은 무엇인가요?",
    "ANSWER": "CHINA"
  },
  {
    "QUESTION": "건지섬의 국가한글명은 무엇인가요?",
    "ANSWER": "그레나다"
  },
  {
    "QUESTION": "홍콩의 국가코드는 무엇인가요?",
    "ANSWER": "홍콩의 국가코드는 HK입니다."
  },
  {
    "QUESTION": "인도네시아의 국가대륙코드는 무엇인가요?",
    "ANSWER": "아시아"
  },
  {
    "QUESTION": "'IN'은 어느 나라의 국가코드인가요?",
    "ANSWER": "인도"
  },
  {
    "QUESTION": "일본의 국가한글명은 무엇인가요?",
    "ANSWER": "일본"
  },
  {
    "QUESTION": "키르기스스탄의 국가영문명은 무엇인가요?",
    "ANSWER": "KYRGYZSTAN"
  },
  {
    "QUESTION": "캄보디아는 어느 대륙에 속해있나요?",
    "ANSWER": "캄보디아는 아시아 대